<a href="https://colab.research.google.com/github/beercafeguy/PyMLLab/blob/master/practice/random_forest_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
path = '/content/drive/MyDrive/sample_data/titanic.csv'
df_orig = pd.read_csv(path)
df_orig.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [66]:
cols = ['PassengerId',
 'Name',
 'SibSp',
 'Pclass',
 'Sex',
 'Age',
 'Parch',
 'Fare',
 'Cabin',
 'Embarked',
 'Survived'
 ]
df = df_orig[cols]
df['Cabin'] = df.Cabin.apply(lambda x:x[0])
df.head()

<ipython-input-66-192f0350370c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cabin'] = df.Cabin.apply(lambda x:x[0])


,PassengerId,Name,SibSp,Pclass,Sex,Age,Parch,Fare,Cabin,Embarked,Survived
0,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,female,38.0,0,71.2833,C,C,1
1,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,female,35.0,0,53.1000,C,S,1
2,7,"McCarthy, Mr. Timothy J",0,1,male,54.0,0,51.8625,E,S,0
3,11,"Sandstrom, Miss. Marguerite Rut",1,3,female,4.0,1,16.7000,G,S,1
4,12,"Bonnell, Miss. Elizabeth",0,1,female,58.0,0,26.5500,C,S,1


In [67]:
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

In [68]:
y

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1])

In [69]:
X[1:5]

array([[1, 'female', 35.0, 0, 53.1, 'C', 'S'],
       [1, 'male', 54.0, 0, 51.8625, 'E', 'S'],
       [3, 'female', 4.0, 1, 16.7, 'G', 'S'],
       [1, 'female', 58.0, 0, 26.55, 'C', 'S']], dtype=object)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Name         183 non-null    object 
 2   SibSp        183 non-null    int64  
 3   Pclass       183 non-null    int64  
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   Parch        183 non-null    int64  
 7   Fare         183 non-null    float64
 8   Cabin        183 non-null    object 
 9   Embarked     183 non-null    object 
 10  Survived     183 non-null    int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 15.9+ KB


In [71]:
# encode categorical variables
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
ohe = OneHotEncoder()
X[:,1] = le.fit_transform(X[:,1])
ct = ColumnTransformer(transformers=[('encoder',ohe,[5,6])],remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

array([[0.0, 0.0, 1.0, ..., 38.0, 0, 71.2833],
       [0.0, 0.0, 1.0, ..., 35.0, 0, 53.1],
       [0.0, 0.0, 0.0, ..., 54.0, 0, 51.8625],
       ...,
       [0.0, 0.0, 1.0, ..., 56.0, 1, 83.1583],
       [0.0, 1.0, 0.0, ..., 19.0, 0, 30.0],
       [0.0, 0.0, 1.0, ..., 26.0, 0, 30.0]], dtype=object)

In [72]:
print(X[1:5])

[[0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1 0 35.0 0 53.1]
 [0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 1 1 54.0 0 51.8625]
 [0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 1.0 3 0 4.0 1 16.7]
 [0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1 0 58.0 0 26.55]]


In [73]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2023)

<146x230 sparse matrix of type '<class 'numpy.float64'>'
	with 1069 stored elements in Compressed Sparse Row format>

In [74]:
from sklearn.ensemble import RandomForestClassifier

classifier_basic = RandomForestClassifier(n_estimators=10)
classifier_basic.fit(X_train,y_train)
classifier_basic

RandomForestClassifier(n_estimators=10)

In [75]:
y_pred = classifier_basic.predict(X_test)

In [76]:
from sklearn.metrics import accuracy_score,roc_auc_score,confusion_matrix
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.6756756756756757
0.6118881118881119
[[ 5  6]
 [ 6 20]]


Model hyperparam tuning

In [78]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(
    estimator=classifier_basic,
    X = X_train,
    y = y_train,
    cv = 10
)

print("Accuracy {:.2f} %".format(accuracies.mean() *100))
print("Standard deviations {:.2f} %".format(accuracies.std() *100))

Accuracy 69.67 %
Standard deviations 11.94 %


In [79]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[5,10,15,20,30,40,50,100],
    'criterion':['gini','entropy']
}
grid_search_cv = GridSearchCV (
    estimator = classifier_basic,
    param_grid = params,
    scoring = 'accuracy',
    cv =10,
    n_jobs=-1
)

grid_search_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(n_estimators=10),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [5, 10, 15, 20, 30, 40, 50, 100]},
             scoring='accuracy')

In [80]:
best_accuracy = grid_search_cv.best_score_
best_params = grid_search_cv.best_params_
print("Best Accuracy {:.2f} %".format(best_accuracy *100))
print("Best Parameters ",best_params)

Best Accuracy 71.76 %
Best Parameters  {'criterion': 'gini', 'n_estimators': 5}


### In pySpark

In [81]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [82]:

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [85]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [86]:
!ls -ltr

total 223380
drwxr-xr-x 13 1000 1000      4096 Feb 22  2021 spark-3.1.1-bin-hadoop3.2
-rw-r--r--  1 root root 228721937 Feb 22  2021 spark-3.1.1-bin-hadoop3.2.tgz
drwxr-xr-x  1 root root      4096 Aug 15 13:37 sample_data
drwx------  6 root root      4096 Aug 17 03:17 drive


In [92]:
from pyspark.sql.functions import *
dataset = (
    spark.read.option('header','true').csv(path)
    .drop('PassengerId','Name','Ticket')
    .withColumn('Cabin',col('Cabin').substr(0,1))
)
dataset.columns

['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Cabin',
 'Embarked']

In [93]:
dataset.show()

+--------+------+------+----+-----+-----+--------+-----+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|    Fare|Cabin|Embarked|
+--------+------+------+----+-----+-----+--------+-----+--------+
|       1|     1|female|38.0|    1|    0| 71.2833|    C|       C|
|       1|     1|female|35.0|    1|    0|    53.1|    C|       S|
|       0|     1|  male|54.0|    0|    0| 51.8625|    E|       S|
|       1|     3|female| 4.0|    1|    1|    16.7|    G|       S|
|       1|     1|female|58.0|    0|    0|   26.55|    C|       S|
|       1|     2|  male|34.0|    0|    0|    13.0|    D|       S|
|       1|     1|  male|28.0|    0|    0|    35.5|    A|       S|
|       0|     1|  male|19.0|    3|    2|   263.0|    C|       S|
|       1|     1|female|49.0|    1|    0| 76.7292|    D|       C|
|       0|     1|  male|65.0|    0|    1| 61.9792|    B|       C|
|       0|     1|  male|45.0|    1|    0|  83.475|    C|       S|
|       1|     2|female|29.0|    0|    0|    10.5|    F|       S|
|       0|

In [100]:
column_ordered = [
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Cabin',
 'Embarked','Survived']
dataset_ready = dataset.select(column_ordered)

cat_columns = ['Sex','Cabin','Embarked']
cat_cols_index = list(map(lambda x:x+'_index',cat_columns))
cat_cols_index

['Sex_index', 'Cabin_index', 'Embarked_index']

In [95]:
dataset_ready.show()

+------+------+----+-----+-----+--------+-----+--------+--------+
|Pclass|   Sex| Age|SibSp|Parch|    Fare|Cabin|Embarked|Survived|
+------+------+----+-----+-----+--------+-----+--------+--------+
|     1|female|38.0|    1|    0| 71.2833|    C|       C|       1|
|     1|female|35.0|    1|    0|    53.1|    C|       S|       1|
|     1|  male|54.0|    0|    0| 51.8625|    E|       S|       0|
|     3|female| 4.0|    1|    1|    16.7|    G|       S|       1|
|     1|female|58.0|    0|    0|   26.55|    C|       S|       1|
|     2|  male|34.0|    0|    0|    13.0|    D|       S|       1|
|     1|  male|28.0|    0|    0|    35.5|    A|       S|       1|
|     1|  male|19.0|    3|    2|   263.0|    C|       S|       0|
|     1|female|49.0|    1|    0| 76.7292|    D|       C|       1|
|     1|  male|65.0|    0|    1| 61.9792|    B|       C|       0|
|     1|  male|45.0|    1|    0|  83.475|    C|       S|       0|
|     2|female|29.0|    0|    0|    10.5|    F|       S|       1|
|     3|  

In [104]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
string_indexer = StringIndexer(inputCols=cat_columns, outputCols=cat_cols_index)
string_indexed = string_indexer.fit(dataset_ready).transform(dataset_ready)
onehotencoder = OneHotEncoder(inputCols=cat_cols_index, outputCols=['Sex_ohe','Cabin_ohe','Embarked_ohe'])
encoded_df = onehotencoder.fit(string_indexed).transform(string_indexed)
string_indexed.show(5)
encoded_df.show(5)

+------+------+----+-----+-----+-------+-----+--------+--------+---------+-----------+--------------+
|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|Survived|Sex_index|Cabin_index|Embarked_index|
+------+------+----+-----+-----+-------+-----+--------+--------+---------+-----------+--------------+
|     1|female|38.0|    1|    0|71.2833|    C|       C|       1|      1.0|        0.0|           1.0|
|     1|female|35.0|    1|    0|   53.1|    C|       S|       1|      1.0|        0.0|           0.0|
|     1|  male|54.0|    0|    0|51.8625|    E|       S|       0|      0.0|        3.0|           0.0|
|     3|female| 4.0|    1|    1|   16.7|    G|       S|       1|      1.0|        6.0|           0.0|
|     1|female|58.0|    0|    0|  26.55|    C|       S|       1|      1.0|        0.0|           0.0|
+------+------+----+-----+-----+-------+-----+--------+--------+---------+-----------+--------------+
only showing top 5 rows

+------+------+----+-----+-----+-------+-----+--------+--